# 导入库

In [ ]:
#!pip install -U torchtext

from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import Multi30k
from typing import Iterable, List

# 导入Multi30k数据集并做基本处理

In [ ]:
SRC_LANGUAGE = 'de'
TGT_LANGUAGE = 'en'

# 定义token的字典, 定义vocab字典
token_transform = {}
vocab_transform = {}

# 创建源语言和目标语言的Tokenizer, 确保依赖关系已经安装
#!pip install -U spacy
!python -m spacy download en
!python -m spacy download de
import de_core_news_sm
import en_core_web_sm

# get_tokenizer是分词函数, 如果没有特殊的则按照英语的空格分割, 如果有这按照对应的分词库返回. 比如spacy, 返回对应的分词库

token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='de')
token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='en')

# 构建生成分词的迭代器

In [ ]:
def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}
    # data_iter: 对象的迭代对象 Multi30k对象
    # language: 对应的翻译语言 {'de': 0, 'en': 1}
    
    for data_sample in data_iter:
        # data_sample:(德文, 英文)
        # data_sample:('Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.\n', 'Two young, White males are outside near many bushes.\n')
        # token_transform['de']()=['Zwei', 'junge', 'weiße', 'Männer', 'sind', 'im', 'Freien', 'in', 'der', 'Nähe', 'vieler', 'Büsche', '.', '\n']
        # or  token_transform['en']分别进行构造对应的字典

        yield token_transform[language](data_sample[language_index[language]])

# 定义特殊字符并下载数据设置默认索引
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3

# 确保标记按其索引的顺序正确插入到词汇表中
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    # 训练数据集的迭代器,
    # 数据集是用英文描述图像的英文语句, 然后人工将其翻译为德文的语句,有两个文件, 一个是train.zh 一个是train.en文件,
    # 然后将其构建为(德文, 英文)的形式
    train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    # 创建torchtext的vocab对象, 即词汇表
    vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(train_iter, ln),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)

# 将 UNK_IDX 设置为默认索引。未找到令牌时返回此索引
# 如果未设置，则在 Vocabulary 中找不到查询的标记时抛出 RuntimeError
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    vocab_transform[ln].set_default_index(UNK_IDX)

# 导入网络搭建所需库

In [ ]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

# 运行设备

In [ ]:
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 定义位置编码器类

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self,emb_size: int,dropout: float, maxlen: int = 5000):
        
        # emb_size: 词嵌入的维度大小
        # dropout: 正则化的大小
        # maxlen: 句子的最大长度
        
        super(PositionalEncoding, self).__init__()
        # 将1000的2i/d_model变型为e的指数形式
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        # 效果等价与torch.arange(0, maxlen).unsqueeze(1)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        # 构建一个(maxlen, emb_size)大小的全零矩阵
        pos_embedding = torch.zeros((maxlen, emb_size))
        # 偶数列是正弦函数填充
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        # 奇数列是余弦函数填充
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        # 将其维度变成三维, 为了后期方便计算
        pos_embedding = pos_embedding.unsqueeze(-2)
        # 添加dropout层, 防止过拟合
        self.dropout = nn.Dropout(dropout)
        
        # 向模块添加持久缓冲区。
        # 这通常用于注册不应被视为模型参数的缓冲区。例如，pos_embedding不是一个参数，而是持久状态的一部分。
        # 缓冲区可以使用给定的名称作为属性访问。
        # 说明：
        # 应该就是在内存中定义一个常量，同时，模型保存和加载的时候可以写入和读出
        
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        # 将token_embedding和位置编码相融合
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# 定义词嵌入层类

In [ ]:
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        
        # vocab_size:词表的大小
        # emb_size:词嵌入的维度
        
        super(TokenEmbedding, self).__init__()
        # 调用nn中的预定义层Embedding, 获取一个词嵌入对象self.embedding
        self.embedding = nn.Embedding(vocab_size, emb_size)
        # 将emb_size传入类内, 变成类内的变量
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        # 让 embeddings vector 在增加 之后的 postion encoing 之前相对大一些的操作，
        # 主要是为了让position encoding 相对的小，这样会让原来的 embedding vector 中的信息在和 position encoding 的信息相加时不至于丢失掉
        # 让 embeddings vector 相对大一些
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# 构建Seq2SeqTransformer模型

In [ ]:
class Seq2SeqTransformer(nn.Module):
    def __init__(self,num_encoder_layers, num_decoder_layers,emb_size,nhead,src_vocab_size,tgt_vocab_size,dim_feedforward = 512,dropout = 0.1):
        
        # num_encoder_layers: 编码器的层数
        # num_decoder_layers: 解码器的层数
        # emb_size: 词嵌入的维度
        # nhead: 头数
        # src_vocab_size: 源语言的词表大小
        # tgt_vocab_size: 目标语言的词表大小
        # dim_feedforward: 前馈全连接层的维度
        # dropout: 正则化的大小
        
        # 继承nn.Module类, 一般继承习惯行的写法
        super(Seq2SeqTransformer, self).__init__()
        # 创建Transformer对象
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        # 创建全连接线性层
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        # 创建源语言的embedding层
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        # 创建目标语言的embedding层
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        # 创建位置编码器层对象
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,src, trg, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, memory_key_padding_mask):
        
        # src: 源语言
        # trg: 目标语言
        # src_mask: 源语言掩码
        # tgt_mask: 目标语言掩码
        # src_padding_mask: 源语言的padding_mask
        # tgt_padding_mask: 目标语言的padding_mask
        # memory_key_padding_mask: 中间语义张量的padding_mask
        
        # 获取源语言的embedding张量融合了位置编码
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        # 获取目标语言的embedding张量融合了位置编码
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        # 经过Transformer进行编解码之后输出out值
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None, src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        # outs值经过输出层得到最后的输出分布值
        return self.generator(outs)
    # 定义Transformer的编码器
    def encode(self, src, src_mask):
        
        # src:源语言
        # src_mask:源语言掩码
        
        return self.transformer.encoder(self.positional_encoding(self.src_tok_emb(src)), src_mask)
    # 定义Transformer的解码器
    def decode(self, tgt, memory, tgt_mask):
        
        # tgt:目标语言
        # memory:中间语言张量输出
        # tgt_mask: 目标语言的掩码
        
        return self.transformer.decoder(self.positional_encoding(self.tgt_tok_emb(tgt)), memory, tgt_mask)

# 定义掩码

作用是防止模型在进行预测的过程中查看到未来的单词. 同时需要掩码来隐藏源语言和目标语言的padding tokens

In [ ]:
def generate_square_subsequent_mask(sz):
    # sz: 句子的长度
    # triu生成的是上三角, 经过transpose之后变成了下三角矩阵
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    # 将0的位置填充负无穷小, 将1的位置填充为0
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def create_mask(src, tgt):
    
    # src: 源语言张量形状为: [seq_length , batch_size]
    # tgt: 目标语言张量形状为: [seq_length , batch_size]
    
    # 获取源语言的句子长度
    src_seq_len = src.shape[0]
    # 获取目标语言的句子长度
    tgt_seq_len = tgt.shape[0]
    # 产生目标语言的掩码张量
    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    # 产生源语言的掩码张量
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)
    # 构建源语言的padding_mask  src_padding_mask==> [batch_size, seq_len]
    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    # 构建目标语言的padding_mask tgt_paddig_mask ==> [batch_size, seq_len-1]
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

# 参数调整

In [ ]:
SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
EMB_SIZE = 1024
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 128
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3
NUM_EPOCHS = 50

model_path = './transformer_translation.pth'

# 初始化GPU

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

# 实例化模型并定义损失函数和优化器

In [ ]:
# 设置种子用于生成随机数，以使得结果是确定的
torch.manual_seed(0)

# 设置调用时候使用的参数
SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
EMB_SIZE = 512 #d_model 1_512
NHEAD = 8 #h 1_8
FFN_HID_DIM = 2048 #d_ff 1_2048
BATCH_SIZE = 128 
NUM_ENCODER_LAYERS = 6 #N 1_6
NUM_DECODER_LAYERS = 6 #N 1_6


# 实例化Transformer对象
transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)



# 为了保证每层的输入和输出的方差相同, 防止梯度消失问题
for p in transformer.parameters():
    if p.dim() > 1:
        # 此处使用的是xavier的均匀分布
        nn.init.xavier_uniform_(p)
        
# 如果有GPU则将模型移动到GPU
transformer = transformer.to(DEVICE)

# 定义损失函数
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)
# 定义优化器  betas: 用于计算梯度及其平方的运行平均值的系数  eps:添加到分母以提高数值稳定性
#optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = torch.optim.Adam(transformer.parameters(), lr=1e-3, betas=(0.9, 0.98), eps=1e-9) # 1_0.0001
#自适应调整学习率
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=False, threshold=0.0001, threshold_mode='rel', cooldown=2, min_lr=0, eps=1e-08)

# 将字符串转化为整数的tensor张量

In [ ]:
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# 在句子首尾添加起始和结束符号

In [ ]:
# 辅助函数, 完成句子首尾BOS/EOS的添加过程
def tensor_transform(token_ids: List[int]):
    # 添加的是列表形式的数据, 将BOS和EOS添加到句子的首尾部分
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

text_transform = {}
# 循环添加源语言和目标语言
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln], #Tokenization
                                               vocab_transform[ln], #Numericalization
                                               tensor_transform) # Add BOS/EOS and create tensor

# 数据进行批次化处理

In [ ]:
# 按照批次进行源语言和目标语言的组装
def collate_fn(batch):
    # 定义源语言和目标语言的批次列表
    src_batch, tgt_batch = [], []
    # 循环批次样本
    for src_sample, tgt_sample in batch:
        # 添加源语言句子到列表中
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
        # 添加目标语言句子到列表中
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))
    # 将源语言和目标语言进行截断补齐  PAD_IDX=1
    # src_batch的形状为: [seq_length, batch]  seq_length是最长的句子长度
    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    # tgt_batch的形状为: [seq_length, batch]  seq_length是最长的句子长度
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

# 定义批次训练函数

In [ ]:
def train_epoch(model, optimizer):
    # 开启训练模式
    model.train()
    # 定义其实的损失值为0
    losses = 0
    # 获取训练数据集的迭代器, 语言对为(zh, en)
    train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    # 加载数据, 按照一个批次一个批次进行加载, 返回一个迭代器
    train_dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)
    # 循环数据迭代器
    for src, tgt in train_dataloader:
        # 将源语言数据移动到对应的设备上去
        src = src.to(DEVICE)
        # 将目标语言数据移动到对应设备上去
        tgt = tgt.to(DEVICE)
        # 获取输入真实的张量 第一个单词到倒数第二个单词
        tgt_input = tgt[:-1, :]
        # 调用mask函数, 生成对应的四个mask
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)
        # 调用模型进行训练, 得到最后的张量分布
        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)
        # 梯度清零
        optimizer.zero_grad()
        # 获取输出真实的标签数据  第二个单词到最后一个单词
        tgt_out = tgt[1:, :]
        # 计算损失
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        # 反向传播
        loss.backward()
        # 梯度更新
        optimizer.step()

        # 损失值累加求和
        losses += loss.item()
    # 返回平均损失值
    return losses / len(train_dataloader)

# 定义批次评估函数

In [ ]:
def evaluate(model):
    # 开启模型评估模式
    model.eval()
    # 定义起始损失值
    losses = 0
    # 加载验证数据集, 语言对为(zh, en)
    val_iter = Multi30k(split='valid', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    # 返回验证集的数据加载器
    val_dataloader = DataLoader(val_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)
    # 循环验证集
    for src, tgt in val_dataloader:
        # 源语言数据移动到对应的设备上
        src = src.to(DEVICE)
        # 目标语言数据移动到对应的设备上
        tgt = tgt.to(DEVICE)
        # 获取输入的真实的张量
        tgt_input = tgt[:-1, :]
        # 调用mask函数, 产生对应的四个mask值
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)
        # 调用模型, 得到对应的输出分布值
        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)
        # 获取输出的真实张量
        tgt_out = tgt[1:, :]
        # 计算损失值
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        # 损失值累加, 求和
        losses += loss.item()
    # 求得对应的平均损失
    return losses / len(val_dataloader)

# 开始训练

In [ ]:
from timeit import default_timer as timer
TLOSS = []
VLOSS = []
TN = []
min_loss = 100000 # 随便设置一个比较大的数

# 循环整个数据集num_epochs次
for epoch in range(1, NUM_EPOCHS+1):
    # 获取开始时间
    start_time = timer()
    # 将整个训练数据集进行训练
    train_loss = train_epoch(transformer, optimizer
    # 获取结束时间
    end_time = timer()
    # 将整个验证集进行评估
    val_loss = evaluate(transformer)
    tn = end_time - start_time
    TLOSS.append(train_loss)
    VLOSS.append(val_loss)
    TN.append(tn)
    scheduler.step(val_loss)

    # 打印每个epoch的训练损失, 验证损失, 和训练时间.
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {tn:.3f}s"))
    if val_loss < min_loss:
        min_loss = val_loss
        print("save model")
        torch.save(transformer.state_dict(), model_path)

# 展示结果

In [ ]:
from matplotlib import pyplot as plt

x=range(0,NUM_EPOCHS)
plt.plot(x,TLOSS,label='train_loss',color='green') #设置折线颜色和标签
plt.plot(x,VLOSS,label='val_loss',color='blue')
plt.xticks(x[::10])
plt.xlabel('epoch')
plt.ylabel('loss')

#绘制网格
plt.grid(alpha=0.4,linestyle='-.')  #alpha设置网格透明度
#添加图例
plt.legend(loc='upper right') #设置图例字体及其位置，可以不写loc参数，默认为右上
plt.savefig("./wmt18_10.jpg")
plt.show()

print(min_loss)
print(sum(TN)/len(TN))
transformer.load_state_dict(torch.load(model_path))
print('加载成功')

# 翻译函数

## 使用贪心算法构建生成序列函数

In [ ]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys

## 定义最终的翻译转化函数

In [ ]:
def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")